In [ ]:
from pathlib import Path

import pandas as pd
from codetf.models import load_model_pipeline
from sacrebleu.metrics import BLEU
from datasets import load_dataset

In [ ]:
root_dir = Path.cwd()

In [ ]:
dataset = load_dataset("json", data_files={
    "test": str(root_dir / "data" / "test.jsonl"),
}, cache_dir=root_dir / "data" / "cache")

In [ ]:
def inference(doc):
    doc["ref"] = [" ".join(docstring) for docstring in doc["docstring_tokens"]]
    return doc    

dataset = dataset.map(inference, batched=True)

In [ ]:
dataset.set_format(type="pandas", columns=["ref", "code"])
df = dataset["test"][:]
df.head()

In [ ]:
model = load_model_pipeline(model_name="codet5", task="sum_python", model_type="base")
preds = []

for i, row in df.head(100).iterrows():
    if i % 100 == 0: print(i)
    preds.extend(model.predict([row["code"]]))

In [ ]:
refs = df["ref"].head(100).tolist()

In [ ]:
bleu = BLEU()
bleu.corpus_score(preds, [refs]).__dict__